In [1]:
print("Hello World")

Hello World


In [2]:
%pip install openai 
%pip install python-dotenv


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
from openai import OpenAI
from dotenv import load_dotenv

Save the OPENAI_KEY into ".env" file, and *** AND .env TO YOUR .gitignore FILE. ***

In [5]:
# from dotenv import load_dotenv
# load_dotenv()  # take environment variables from .env.

True

For normal use GPT-4 test

In [10]:
PROMPT = f"""You are provided a calendar dataset. This calendar is a Pandas dataframe named filtered_calendar_data, columns = [ID, status, summary, start, end , duration, attendees].
    This DataFrame filtered_calendar_data is a filtered dataframe, which includes all of your meeting records you need to answer the question. For example, if the question is in which day I have most meetings this week, then the calendar_data only consists of this weeks meeting records.
    Provide a chain of thought for your reasoning, analyze the date requirement, filter dataframe, and then fill the python function <query_dataframe>. 
    First, based on the dataframe and question, breakdown the question into several steps; Second, implement those steps in Python using the datetime module and Pandas:

    The input dataframe filtered_calendar_data have following columns:
    - ID: meeting ID;
    - status: meeting status, including the following status: cancelled, confirmed, tentative, data type 'string'.
    - summary: meeting or event topic, data type 'string';
    - start: the start datetime of meeting, data type 'datetime64[ns]', and date format: '%Y-%m-%d %H:%M:%S'.;
    - end: the start datetime of meeting, data type 'datetime64[ns]', and date format: '%Y-%m-%d %H:%M:%S',;
    - duration: meeting duration, data type 'int64', and date format: second;
    - attendees: email address of people who attend the meeting, data type 'str', emails address are delimited by the line terminator.
    
    **instruction**
    - provided python packages:'pandas' and 'datetime';
    - the python function is <query_dataframe> and the inpout is dataframe 'filtered_calendar_data';
    - all the questions use same timezone, thus don't have to convert the timezone;
    - if the today's date is required, use the floowing python code:
    ```python
    # get today's date, if the question is related to datetime, use today_date, otherwise ignore this code
    today_date = pd.to_datetime('2023-10-02 09:25:30', format='%Y-%m-%d %H:%M:%S')
    ```

    Ensure any code you provide can be executed with all required input and variables defined. 
    The output should be a markdown python code snippet formatted in the following schema, including the leading and trailing "```python" and "```":
    ```python
    def query_dataframe(filtered_calendar_data):

        <your code here>
        
        return answer
    ```    
    
    Here is the question: analyze my calender this week and tell me how I can optimize it to have 30min walk time daily
    """

In [11]:
MODEL = "gpt-4"
# MODEL = "gpt-3.5-turbo"

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content":PROMPT,
        }
    ],
    model=MODEL,
)

# verify the output
generated_text = chat_completion.choices[0].message.content
print(generated_text)

Your issue lies with the fact that `exec()` doesn't have access to the outer scope, thus causes the variable `pd` inside the script to not be defined as it is outside the script. To remedy this, you can pass a dictionary to `exec()` to give it access to certain variables. This way `pd`, `calendar_data` or any other modules or variables that are required, are imported and defined in the appropriate manner before calling the script.

Here is how you could structure it:
```python
import pandas as pd

def query(calendar_data):
    today = pd.to_datetime('2024-03-12')
    calendar_data['start'] = pd.to_datetime(calendar_data['start'])
    calendar_data['end'] = pd.to_datetime(calendar_data['end'])
    today_events = calendar_data[(calendar_data['start'].dt.date == today) | (calendar_data['end'].dt.date == today)]
    return len(today_events)

# Define whatever your calendar data is
calendar_data = <Whatever your real calendar data is>

# Define the current namespace
namespace = {"pd": pd, "

Prompt for calendar questions

In [13]:
# prompt = """ You will be provided with a dataframe "calendar_data". "calendar_data" consists of 7 columns, which can be represented as calendar_Data(ID, status, summary, start, end , duration, attendees).
# Input dataset: ID is the meeting ID; start is the meeting start time; end is the meeting end time; duration is how long the meeting last; attendees is who will attend the meeting

# Write 10 different questions to operate dataframe. For example:

# "create meeting with Natalia Shreve tomorrow at 10 am",
# "remove tomorrow's meeting at 10 am",
# "reschedule recurrent daily Meeting with Bob to 5 pm".
# "Do I have meeting today"
# """

prompt = f"""You are provided a calendar. This calendar is a Pandas dataframe named calendar_data, columns = [ID, status, summary, start, end , duration, attendees].
This DataFrame calendar_data includes all of your meeting schedule.
Your task is generate 30 questions to query this dataframe. Make sure the generated questions cover the fo9llowing operations for calendar data including: Filter, Aggregate, Group, Sort, Compute, and Sub query

Output a json object that contains the following keys: question, answer. Fill the answer key blank.

The input have following columns:
- ID: meeting ID;
- status: meeting status, including the following status: cancelled, confirmed, tentative; cancelled means that meeting is cancelled.
- summary: meeting or event topic;
- start: the start date of meeting, date format: YYYY-MM-DD hh:mm:ss.fff-zz:xx. for example "2024-02-05 12:00:00-00:00";
- end: the start date of meeting, date format: YYYY-MM-DD hh:mm:ss.fff-zz:xx, for example "2024-02-05 13:00:00-00:00";
- duration: meeting duration (second);
- attendees: people who attend the meeting delimited by the line terminator within 1 sentence.

Output JSON: <json with question and answer>
"""

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model=MODEL,
)

# verify the output
generated_text = chat_completion.choices[0].message.content
print(generated_text)

[
{
"question": "What is the total number of meetings and the average duration in seconds?",
"answer": "calendar_data['duration'].agg(['count', 'mean'])"
},
{
"question": "Tell me the total number of meetings by status.",
"answer": "calendar_data.groupby('status')['ID'].count()"
},
{
"question": "Which meeting has the longest duration?",
"answer": "calendar_data[calendar_data['duration'] == calendar_data['duration'].max()]['summary']"
},
{
"question": "What is the shortest meeting duration?",
"answer": "calendar_data['duration'].min()"
},
{
"question": "How many confirmed meetings are scheduled for tomorrow?",
"answer": "calendar_data[(calendar_data['status'] == 'confirmed') & (calendar_data['start'].dt.date == pd.to_datetime('tomorrow').date())]['ID'].count()"
},
{
"question": "Sort the meetings by start date in ascending order.",
"answer": "calendar_data.sort_values(by='start')"
},
{
"question": "What are the top 5 meetings with the highest attendance (assuming attendees column are c

[ 
  {"question":"How many meetings were scheduled?", "answer":""},
  {"question":"How many meetings got cancelled?", "answer":""},
  {"question":"How many meetings got confirmed?", "answer":""},
  {"question":"Which meeting had the maximum duration?", "answer":""},
  {"question":"Which meeting had the minimum duration?", "answer":""},
  {"question":"What is the average duration of the meetings?", "answer":""},
  {"question":"What is the total duration of all the meetings?", "answer":""},
  {"question":"What is the start date of the earliest meeting?", "answer":""},
  {"question":"What is the end date of the last meeting?", "answer":""},
  {"question":"Sort the meetings by duration in ascending order.", "answer":""},
  {"question":"Sort the meetings by duration in descending order.", "answer":""},
  {"question":"List all the meetings scheduled for date x.", "answer":""},
  {"question":"List all the attendees for meeting with id x.", "answer":""},
  {"question":"Find the meeting id with summary x.", "answer":""},
  {"question":"List all the meetings with more than x attendees.", "answer":""},
  {"question":"Which meeting had the maximum attendees?", "answer":""},
  {"question":"Which meeting had the minimum attendees?", "answer":""},
  {"question":"Show status of meeting with id x.", "answer":""},
  {"question":"Show the summary of meeting with id x.", "answer":""},
  {"question":"List all the attendees for meetings scheduled on date x.", "answer":""},
  {"question":"What is the status of meetings scheduled for date x?", "answer":""},
  {"question":"Calculate total duration of meetings scheduled on date x.", "answer":""},
  {"question":"List all meetings confirmed for date x.", "answer":""},
  {"question":"List all meetings cancelled for date x.", "answer":""},
  {"question":"Find the duration of meeting with id x.", "answer":""},
  {"question":"Find the end date and time of meeting with id x.", "answer":""},
  {"question":"List the id of all meetings scheduled today.", "answer":""},
  {"question":"List the id of all meetings scheduled tomorrow.", "answer":""},
  {"question":"Find the start date and time of meeting with id x.", "answer":""},
  {"question":"Which person has the most meetings to attend?", "answer":""}
]

In [ ]:
# convert output into json

Prompt for generting meeting date with format YYYY-MM-DD hh:mm:ss-00:00

In [45]:
prompt = """ Help me generate 20 meeting date, including start date and end date. 
Date format: YYYY-MM-DD hh:mm:ss-00:00

Limitations:
The meeting usually lasts 30 minutes to 2 hours;
The meeting should start from 2024-02-03 to 2024-04-05
Every tuesday, there are at least 2 meetings


Here is the example: 
{Start: 2024-03-24 10:00:00-00:00
 End:   2024-03-24 10:45:00-00:00   
}

Results:
"""

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model=MODEL,
)

# verify the output
generated_text = chat_completion.choices[0].message.content
print(generated_text)

1. {Start: 2024-02-06 09:00:00-00:00, End: 2024-02-06 10:30:00-00:00}
2. {Start: 2024-02-06 14:00:00-00:00, End: 2024-02-06 15:30:00-00:00}
3. {Start: 2024-02-13 09:00:00-00:00, End: 2024-02-13 10:30:00-00:00}
4. {Start: 2024-02-13 14:00:00-00:00, End: 2024-02-13 15:30:00-00:00}
5. {Start: 2024-02-20 09:30:00-00:00, End: 2024-02-20 11:00:00-00:00}
6. {Start: 2024-02-20 14:00:00-00:00, End: 2024-02-20 15:30:00-00:00}
7. {Start: 2024-02-27 09:00:00-00:00, End: 2024-02-27 10:30:00-00:00}
8. {Start: 2024-02-27 14:00:00-00:00, End: 2024-02-27 15:30:00-00:00}
9. {Start: 2024-03-05 09:30:00-00:00, End: 2024-03-05 11:00:00-00:00}
10. {Start: 2024-03-05 14:00:00-00:00, End: 2024-03-05 15:30:00-00:00}
11. {Start: 2024-03-12 09:00:00-00:00, End: 2024-03-12 10:30:00-00:00}
12. {Start: 2024-03-12 14:00:00-00:00, End: 2024-03-12 15:30:00-00:00}
13. {Start: 2024-03-19 09:30:00-00:00, End: 2024-03-19 11:00:00-00:00}
14. {Start: 2024-03-19 14:00:00-00:00, End: 2024-03-19 15:30:00-00:00}
15. {Start: 202

Prompt for generting meeting topic and attendees

In [43]:
prompt = """ You are walter, a new software engineer employee of NinjaTech AI. You have many meetings with team members, 
for example  meeting with the team leader Arash about the project progress and research plan. You have 4 team members: Alice, Bob, Carrie, David. You have regular meeting every Monday about project progress.
You also need to talk to them about research issues. 
Help me generate all your 10 meetings, in terms of meeting topic and attendees. 
Meeting attendees are their ninjatech emails, for exampole ${name}$@ninjatech.ai(${status}$).  
${name}$ is their name and ${status}$ denotes if they accepted the meeting invitation. ${status}$ could be accepted, needsAction, and rejected.
Each meeting usually include 2 people but no more than the all people in this company.
The output format is ${name}$@ninjatech.ai(${status}$)\n ${name}$@ninjatech.ai(${status}$)
Here is the example: 
"meeting Topic: 
temp-testing@ninjatech.ai(accepted)\n yevhenia.spiridonova@ninjatech.ai(needsAction)"
"""

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model=MODEL,
)

# verify the output
generated_text = chat_completion.choices[0].message.content
print(generated_text)

Sure, let's generate these 10 meetings:

1. Meeting Topic: Project Kickoff 
walter@ninjatech.ai(accepted)
 arash@ninjatech.ai(accepted)

2. Meeting Topic: Agile Development Workflow
walter@ninjatech.ai(accepted)
 alice@ninjatech.ai(accepted)

3. Meeting Topic: Code Review 
walter@ninjatech.ai(accepted)
 bob@ninjatech.ai(accepted)

4. Meeting Topic: Research Planning
walter@ninjatech.ai(accepted)
 carrie@ninjatech.ai(accepted)

5. Meeting Topic: Debugging And Error Handling 
walter@ninjatech.ai(accepted)
 david@ninjatech.ai(accepted)

6. Meeting Topic: Weekly Progress Review
walter@ninjatech.ai(accepted)
 arash@ninjatech.ai(needsAction)

7. Meeting Topic: Database Architecture 
walter@ninjatech.ai(accepted)
 alice@ninjatech.ai(needsAction)

8. Meeting Topic: Security Best Practices 
walter@ninjatech.ai(accepted)
 bob@ninjatech.ai(accepted)

9. Meeting Topic: UI/UX Design 
walter@ninjatech.ai(accepted)
 carrie@ninjatech.ai(rejected)

10. Meeting Topic: API Integration 
walter@ninjatech.a

In [ ]:
prompt = """ You are walter, a new software engineer employee of NinjaTech AI. You have many meetings team members, 
for example meet with organizer Natalia about the job application, interview process, contract discussion and meeting with the team leader Arash. 
Help me generate all your 10 meetings, in terms of meeting topic and attendees. 
Meeting attendees are their ninjatech emails, for exampole ${name}$@ninjatech.ai(${status}$).  
${name}$ is their name and ${status}$ denotes if they accepted the meeting invitation. ${status}$ could be accepted, needsAction, and rejected.
Each meeting usually include 2 people but no more than the all people in this company.
The output format is ${name}$@ninjatech.ai(${status}$)\n ${name}$@ninjatech.ai(${status}$)
Here is the example: 
"meeting Topic: 
temp-testing@ninjatech.ai(accepted)\n yevhenia.spiridonova@ninjatech.ai(needsAction)"
"""

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model=MODEL,
)

# verify the output
generated_text = chat_completion.choices[0].message.content
print(generated_text)

Prompt for generating attendees

In [36]:
prompt = """ You are a meeting organzer. Help me generate 7 meeting start date and end data for this week. Today is 2024-03-27 Wednesday.
The meeting may be in several days age, may be in today, and may be several days late. For each day, there are no meeting, 1 meeting or 2 meeting.
The meeting usually lasts for 10 minutes to 2 hours.
Date format: YYYY-MM-DD hh:mm:ss-00:00. The meeting should be in time order
Here is the example: 
{Start date: 2024-03-27 12:00:00-00:00
 end date: 2024-03-27 12:30:00-00:00
}
"""

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model=MODEL,
)

# verify the output
generated_text = chat_completion.choices[0].message.content
print(generated_text)

Sure, you can use the `json` module provided by Python to achieve this. Below is a simple function that converts a multiline string into a .json format:

```python
import json

def convert_to_json(multiline_string):
    # Split the multiline string into a list of lines
    lines = multiline_string.split("\n")
    
    # Convert the list into a json string
    json_string = json.dumps(lines)
    
    return json_string

# Test the function
multiline_string = """
This is line 1
This is line 2
This is line 3
"""

print(convert_to_json(multiline_string))
```

Please note that this code would convert each line in the multiline string as a separate item of an array in the resulting .json.

If your multiline string is actually representing a .json, you don't need to split your string line by line. You can directly load it into .json as shown below:

```python
import json

def convert_to_json(multiline_json_string):
    try:
        json_data = json.loads(multiline_json_string)
        return 

Verify the ***gpt-3.5-turbo***

In [12]:
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    # {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    # {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
    {"role": "user", "content": "Are you GPT4 or GPT3.5?"}
  ]
)

print(completion.choices[0].message.content)

I am an instance of OpenAI's GPT-3 model.
